In [ ]:
from Classes.Person import Person
import pickle
import numpy as np
import math
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
honest = pickle.load(open("batch_run_honest_1","rb"))
strategic = pickle.load(open("batch_run_strategic_1","rb"))

In [ ]:
def getFluctuations(pollList):
    fluctuation = {}
    length = len(pollList)
    for cand in pollList[0]:
        fluctuation.update({cand: 0})
                    
    for cand in fluctuation:
        differences = []
        for pollId in range(length-1):
            differences.append(pollList[pollId + 1].get(cand) - pollList[pollId].get(cand))
        fluctuation.update({cand:(sum(differences) / length)})

    return fluctuation

In [ ]:
def convertRuns(_data):
    runs = pd.DataFrame(columns=['run_id', 'cand_unique_id', 'cand_votes', 'cand_position', 'strat_perc'])
    for i, run in _data.iterrows():
        cand_fluctuations = getFluctuations(run.Polls)
        for cand in run.Polls[len(run.Polls)-1]:
            runs = runs.append({'run_id':i,'cand_unique_id':cand.unique_id,'cand_votes':run[1][0].get(cand),'cand_position':cand.position,'strat_perc': run[3],'fluctuation':cand_fluctuations.get(cand)},ignore_index=True)
    runs['cand_votes'] = runs['cand_votes'].convert_dtypes(int)
    return runs

In [ ]:
honest_runs = convertRuns(honest)

In [ ]:
strategic_runs = convertRuns(strategic)

In [ ]:
honest, strategic = None, None

In [ ]:
def addDistance(df:pd.DataFrame):
    df['distance'] = df['cand_position'].apply(lambda x: np.linalg.norm([1.0,1.0] - x))
    return df

In [ ]:
honest_runs = addDistance(honest_runs)
display(honest_runs.sort_values(by=['distance']))
display(honest_runs.sort_values(by=['cand_votes'],ascending=False))

strategic_runs = addDistance(strategic_runs)
display(strategic_runs.sort_values(by=['distance']))
display(strategic_runs.sort_values(by=['cand_votes'],ascending=False))

In [ ]:
display(sns.pairplot(honest_runs[['cand_votes','distance']]))
display(sns.pairplot(strategic_runs[['cand_votes','distance','strat_perc','fluctuation']]))

In [ ]:
# display(honest_runs[['cand_votes']].describe(include='all'))
# display(honest_runs[['cand_votes']].plot.box())
# display(strategic_runs[['cand_votes']].describe(include='all'))
# display(strategic_runs[['cand_votes']].plot.box())

# Effect van plurality voting op mensen dat niet op hun eerste voorkeur kiest.

Hier laten we een scatter plot zien met de gemiddelde fluctuatie in stemmen per pijlingsronden.
Wat hier uit te zien is is dat de kans dat je stemmen toe krijgt van strategische stemmers later in het stem proces steeds kleiner wordt deste verder je van het midden van het politeke centrum af staat.
<br><br>
De X-as van dit model laat de afstand van een kandidaad naar het centrum zien.<br>
De Y-as laat hier de hoeveelheid stemmen zien die een kandidaat krijg/verliest gemiddeld per pijling.

In [ ]:
plt.figure()
plt.scatter(data=strategic_runs, x='distance',y='fluctuation')
plt.hlines(y=0, xmin = 0.0, xmax = 1.4, color='r')
plt.show()

Ons model gebruikt meerdere parameters om te bepalen wanneer een stemmer een strategische stem wilt uitbrengen.
<br>
Hier is goed te zien dat er soms nog wat uitschieters zijn en er veel stemmers strategisch gaan stemmen, maar veel belangrijker dat de mediaan mooi rond de 10% ligt.
<br>
Zoals uit bronnen naar voren komt is het uiteidenlijke stemgedrag van mensen 9% strategisch.

In [ ]:
strategic_runs[['strat_perc']].describe()

De frequentie verdeling van fluctuatie.<br>

In [ ]:
count = strategic_runs.groupby([pd.cut(strategic_runs['distance'],12)])[['fluctuation']].count().reset_index()
count[['fluctuation']].plot.bar(stacked=True)

Hier is een Barplot te zien waar een gemiddelde waarde van de fluctuatie te zien is in verschillende bins.<br>
We hebben gekozen voor 12 bins.<br>
Zo is er duidelijk te zien dat een kandidaad die dichter bij het centrum zit over het algemeen meer stemmen krijgt mochten mensen besluiten strategisch te gaans stemmen.<br>
Het omkeer punt zit rond de 0.7 afstand.

In [ ]:
sums = strategic_runs.groupby([pd.cut(strategic_runs['distance'],12)])[['fluctuation']].mean().reset_index()
sums.plot.bar(x='distance',y='fluctuation',rot=90)

# Effect van pijlingen op het stemgedrag van stemmers met plurality voting.

In [ ]:
raw_poll_1 = pickle.load(open("batch_run_strategic_2_1poll","rb"))
raw_poll_2 = pickle.load(open("batch_run_strategic_2_2poll","rb"))
poll_6 = strategic_runs
raw_poll_14 = pickle.load(open("batch_run_strategic_2_14poll","rb"))

In [ ]:
poll_1 = convertRuns(raw_poll_1)
poll_1 = addDistance(poll_1)


poll_2 = convertRuns(raw_poll_2)
poll_2 = addDistance(poll_2)

poll_14 = convertRuns(raw_poll_14)
poll_14 = addDistance(poll_14)

Om het effect van pijlingen te bekijken op het stemgedrag van .......

In [ ]:
poll_1_mean = poll_1.groupby([pd.cut(poll_1['distance'],5)])[['cand_votes']].mean().reset_index()
poll_1_mean.plot.bar(x='distance',y='cand_votes',rot=90)

In [ ]:
poll_2_mean = poll_2.groupby([pd.cut(poll_2['distance'],5)])[['cand_votes']].mean().reset_index()
poll_2_mean.plot.bar(x='distance',y='cand_votes',rot=90)

In [ ]:
poll_6_mean = poll_6.groupby([pd.cut(poll_6['distance'],5)])[['cand_votes']].mean().reset_index()
poll_6_mean.plot.bar(x='distance',y='cand_votes',rot=90)